# 江小婉心理咨询交互对话

使用完整的三种记忆系统与江小婉进行心理咨询对话。

In [ ]:
import httpx
import json
from IPython.display import display, Markdown

# 配置
POC_URL = "http://localhost:8080"
USER_ID = "xiaowan_full"
SESSION_ID = "notebook_session_001"
DATASET_NAMES = ["kb_psyc"]
ROLE = "psychology_counselor"

In [ ]:
async def chat(message: str):
    """发送消息并获取回复"""
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.post(
            f"{POC_URL}/api/v1/test/conversation",
            json={
                "user_id": USER_ID,
                "session_id": SESSION_ID,
                "message": message,
                "dataset_names": DATASET_NAMES,
                "role": ROLE
            }
        )
        return response.json()

def display_response(result):
    """显示回复"""
    # 咨询师回复
    ai_response = result.get('response', '')
    display(Markdown(f"### 🧑‍⚕️ 陈老师\n\n{ai_response}"))
    
    # 记忆系统使用情况
    context = result.get('context', {})
    
    display(Markdown("---"))
    display(Markdown("### 📊 记忆系统使用情况"))
    
    # Memobase
    profile_status = context.get('user_profile_status', '未知')
    display(Markdown(f"**🎭 Memobase 用户画像**: {profile_status}"))
    
    # Mem0
    mem_count = context.get('session_memories_count', 0)
    mem_status = context.get('session_memories_status', '未知')
    display(Markdown(f"**💭 Mem0 会话记忆**: {mem_status}"))
    
    # Cognee
    kb_count = context.get('knowledge_count', 0)
    kb_status = context.get('knowledge_status', '未知')
    display(Markdown(f"**📚 Cognee 知识库**: {kb_status}"))

## 开始对话

### 第1轮：初次见面

In [ ]:
# 江小婉的消息
message = "你好老师，我是江小婉，最近学习压力特别大"
display(Markdown(f"### 🙋 小婉\n\n{message}"))

# 获取回复
result = await chat(message)
display_response(result)

### 第2轮：继续对话

In [ ]:
# 江小婉的消息
message = "主要是数学和物理成绩不好...我很努力了，但还是考不好"
display(Markdown(f"### 🙋 小婉\n\n{message}"))

# 获取回复
result = await chat(message)
display_response(result)

### 查看详细上下文

In [ ]:
# 查看完整的用户画像
context = result.get('context', {})
profile = context.get('user_profile', {})

display(Markdown("## 🎭 Memobase 用户画像详情"))
for key, value in profile.items():
    display(Markdown(f"- **{key}**: {value}"))

# 查看会话记忆
display(Markdown("## 💭 Mem0 会话记忆详情"))
memories = context.get('session_memories', [])
for i, mem in enumerate(memories, 1):
    content = mem.get('content', '')
    display(Markdown(f"{i}. {content}"))

# 查看知识库检索
display(Markdown("## 📚 Cognee 知识库详情"))
knowledge = context.get('knowledge', [])
for i, item in enumerate(knowledge, 1):
    content = item.get('content', '')
    score = item.get('score', 0)
    display(Markdown(f"{i}. (相关度: {score:.2f}) {content[:200]}..."))